<a href="https://colab.research.google.com/github/gibsonx/CE888/blob/master/Federated_Learning_DockerPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Modified by Gibson Xue on July 25, 2021
!pip uninstall --yes tensorflow tensorboard tb-nightly
!pip install --quiet tensorflow==2.5.0
!pip install --quiet tensorflow-federated==0.19.0
!pip install --quiet nest-asyncio
!pip install --quiet tensorboard  # or tensorboard, but not both
!pip install keras
import tensorflow as tf

import nest_asyncio
nest_asyncio.apply()
print(tf.__version__)

from tensorflow_federated.python.simulation import FileCheckpointManager
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd

Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
Uninstalling tensorboard-2.5.0:
  Successfully uninstalled tensorboard-2.5.0
     |████████████████████████████████| 454.3MB 38kB/s 
     |████████████████████████████████| 6.0MB 30.0MB/s 
     |████████████████████████████████| 604kB 15.6MB/s 
     |████████████████████████████████| 194kB 30.6MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 174kB 64.4MB/s 
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: pymc3 3.11.2 has requirement cachetools>=4.2.1, but you'll have cachetools 3.1.1 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement tqdm>=4.36.1, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
2.5.0


In [2]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])

def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [3]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In [142]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
type(example_dataset)
# print(type(example_dataset))

# import tensorflow_datasets as tfds
# df = tfds.as_dataframe(example_dataset.take(1))
# a = df['pixels'].values
# data = np.asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
a
# np.savetxt("foo.csv", a, delimiter=",")
# import numpy as geek

# list_train = []
# len(emnist_train.client_ids)
# for i in range(0,1):
#   client_dataset = emnist_train.create_tf_dataset_for_client(
#       emnist_train.client_ids[i])
#   print(client_dataset.element_spec)
#   for example in client_dataset:
#     pixels = example['pixels'].numpy()
#     label = geek.array_str(example['pixels'].numpy())
#     list_train.append({'pixels': pixels, 'label': label, 'client_id': i, 'is_checked': 0 })
    # # t = geek.array(g)
    # print(pixels)
    # break

# list_train
# df = pd.DataFrame(list_train)
# df.to_csv('data.csv',index=False)
# compression_opts = dict(method='zip',archive_name='data.csv')
# df.to_csv(data.csv, index=False,
#           compression=compression_opts)

array([array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        ,

In [132]:
# df = pd.read_csv('data.csv')
# dataset = tf.data.experimental.CsvDataset('data.csv',[tf.int32],select_cols=[1])


d = tf.convert_to_tensor(data,dtype=tf.float32)
d
# for i in dataset:
#   print(i)
# number_column = df.loc[:,'pixels'].apply(lambda x : tf.convert_to_tensor(x, dtype=tf.float32))
# number_column
# df = tf.data.experimental.make_csv_dataset('data.csv',batch_size=2)
# for i in number_column:
#   print(i)
#   break
# example_dataset = emnist_train.create_tf_dataset_for_client(
#     emnist_train.client_ids[0])

# example_dataset

# preprocessed_example_dataset = preprocess(df.to_numpy())

# preprocessed_example_dataset = preprocess(example_dataset)

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]], dtype=float32)>

In [ ]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [ ]:
state = iterative_process.initialize()
federated_train_data_1 = make_federated_data(emnist_train, sample_clients_1)
state, metrics = iterative_process.next(state, federated_train_data_1)
state